In [1]:
import pandas as pd
import numpy as np
import glob

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
import math

In [5]:
mini_bins=[0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,2];

In [6]:
bin_widths=[mini_bins[i]-mini_bins[i-1] for i in xrange(1,len(mini_bins))]

In [7]:
minikinetic = set(glob.glob("Events6/mini_*")) - set(glob.glob("Events6/mini_baryonic*"))

In [8]:
def calc_kin_energy(arr):
    return arr[4]-math.sqrt(arr[4]**2-(arr[1]**2+arr[2]**2+arr[3]**2))

In [8]:
test=sum_csv=pd.read_csv('Events6/summary.dat',header=None,names=range(6),sep=' ',skipinitialspace=True).values

In [15]:
a=np.array([  3.50000000e-01,   1.70000000e-01,   3.30652064e+00,
         6.87956926e-01,   2.25122717e-01,   9.20554228e-02,
         4.80930574e-02,   2.53742512e-02,   1.13102282e-02,
         6.19603808e-03,   5.11419016e-03,   3.24554375e-03,
         5.90098864e-03])

In [16]:
a[2:]/bin_widths

array([  6.61304128e+01,   1.37591385e+01,   4.50245434e+00,
         1.84110846e+00,   9.61861148e-01,   5.07485024e-01,
         2.26204564e-01,   1.23920762e-01,   1.02283803e-01,
         6.49108750e-02,   4.06964734e-03])

In [9]:
def bin_events(event_file_list,summary_file):
    files = dict()
    for f in event_file_list:
        with open(f) as fs:
            files[float(fs.readline().split()[1])] = f
    
    sum_csv=pd.read_csv(summary_file,header=None,names=range(6),sep=' ',skipinitialspace=True).values
    for i in xrange(len(sum_csv)):
        if sum_csv[i,0] != 'Run':
            continue
        else:
            run_num = float(sum_csv[i,1])
            #print run_num
        sum_num=0;
        MV=0;
        MX=0;
        for j in xrange(i,len(sum_csv)):
            if sum_csv[j,0]=="Total":
                sum_num = sum_csv[j,3]
                MV = sum_csv[j,1]
                MX = sum_csv[j,2]
                #print(sum_num,MV,MX)
                break
        if run_num not in files:
            continue
        event_file=files[run_num]
    
        event_csv=pd.read_csv(event_file,header=None,names=[0,1,2,3,4],sep=' ',skipinitialspace=True).values
        
        par=event_csv[(event_csv[:,0]=="proton")]
        par2=[calc_kin_energy(arr) for arr in par]
        par2=[a for a in par2 if not math.isnan(a)]
        vals,bins=np.histogram(par2,bins=mini_bins)
        
        nar=event_csv[(event_csv[:,0]=="neutron")]
        nar2=[calc_kin_energy(arr) for arr in nar]
        nar2=[a for a in nar2 if not math.isnan(a)]
        nvals,bins=np.histogram(nar2,bins=mini_bins)
        
        vals = vals/bin_widths
        nvals = nvals/bin_widths
        
        try:
            scale=sum_num/float(len(par)+len(nar))
        except ZeroDivisionError:
            print event_file
            scale=1.0
        yield np.append([MV,MX],(vals+nvals)*scale)

In [20]:
kinetic_generator=bin_events(minikinetic,'Events6/summary.dat')

In [21]:
out=[x for x in kinetic_generator]

In [17]:
len(set(glob.glob("Events_fit/mini_baryonic*")))

95

In [18]:
baryonic_generator=bin_events(set(glob.glob("Events_fit/mini_baryonic*")),'Events_fit/summary_baryonic.dat')

In [19]:
outb=[x for x in baryonic_generator]

In [194]:
"""
def bin_events(event_file,summary_file):
    event_csv=pd.read_csv(event_file,header=None,names=[0,1,2,3,4],sep=' ',skipinitialspace=True).values
    run_number=event_csv[0,1]
    sum_csv=pd.read_csv(summary_file,header=None,names=range(6),sep=' ',skipinitialspace=True).values
    sum_arr=[]
    if len(np.where(sum_csv==run_number)[0])<1:
        delete_arr.append([event_file])
        return -1
    else: 
        return 0
    for i in xrange(len(sum_csv))+np.where(sum_csv==run_number)[0][0]:
        if sum_csv[i,0]=="Total":
            sum_num = sum_csv[i,3]
            MV = sum_csv[i,1]
            MX = sum_csv[i,2]
            break
    par=event_csv[(event_csv[:,0]=="proton")]
    par2=[calc_kin_energy(arr) for arr in par]
    par2=[a for a in par2 if not math.isnan(a)]
    vals,bins=np.histogram(par2,bins=mini_bins)
    nar=event_csv[(event_csv[:,0]=="neutron")]
    nar2=[calc_kin_energy(arr) for arr in nar]
    nar2=[a for a in nar2 if not math.isnan(a)]
    nvals,bins=np.histogram(nar2,bins=mini_bins)
    try:
        scale=sum_num/float(len(par)+len(nar))
    except ZeroDivisionError:
        print event_file
        scale=1.0
    return np.append([MV,MX],(vals+nvals)*scale)
"""

In [50]:
#outb=[bin_events(file_name,'Events2/summary_baryonic.dat') for file_name in glob.glob("Events2/mini_baryonic*")]

In [14]:
np.savetxt("miniboone_baryonic_binned_kappa_0_alphaprime_0.00001.csv",outb,delimiter=" ")

In [25]:
np.savetxt("miniboone_binned_kappa_0.001_alphaprime_0.5.csv",out,delimiter=" ")

In [20]:
np.savetxt("miniboone_binned_3ratio_2.csv",outb,delimiter=" ")

In [26]:
np.savetxt("miniboone_binned_3ratio_alphaprime_0.5.csv",out,delimiter=" ")

In [6]:
a=pd.read_csv('Events2/events.dat',header=None,names=[0,1,2,3,4],sep=' ',skipinitialspace=True).values

In [7]:
b=pd.read_csv('Events2/summary.dat',header=None,names=range(6),sep=' ',skipinitialspace=True).values